In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import um_dynamics.robotdynamics as u2d
import numpy as np
import casadi as ca

In [4]:
alpha = u2d.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
base_T0 = [0.190, 0.000, -0.120, 3.142, 0.000, 0.000] #floating base mount
base_T1 = [0.0, 0.000, 0.0, 0.0, 0.000, 0.000] #fixed base mount
kinematic_dict, K, D, P, L = alpha.build_model(root, tip, floating_base=True)
inertial_origins_params, m_params, I_params, g, q, q_dot, base_pose, world_pose = kinematic_dict['parameters']
n_joints = alpha.get_n_joints(root, tip)

In [5]:
L.size(), K.size(), P.size(), D.size()

((1, 1), (1, 1), (1, 1), (4, 4))

In [6]:
D_f = ca.Function('inertia_matrix', [ca.vertcat(*inertial_origins_params), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               q,
                               base_pose,
                               world_pose ],[D])

In [7]:
robot_inertia_matrix = D_f([5e-3 ,-1e-3 ,16e-3, 0, 0, 0,
     5e-3, -1e-3, 16e-3, 0, 0, 0,
     5e-3, -1e-3, 16e-3, 0, 0, 0,
     5e-3, -1e-3, 16e-3, 0, 0, 0],
    [0.341, 0.341, 0.341, 0.341],
    [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0],
    [ca.pi, ca.pi, ca.pi, ca.pi],
    [0.190, 0.000, -0.120, 3.142, 0.000, 0.000],
    [0.0, 1.0, 0, 0, 0, 0])
robot_inertia_matrix

DM(
[[0.0403083, -0.000380482, 8.61616e-05, 0.0100089], 
 [-0.000380482, 0.0523742, -0.0239314, -0.000320027], 
 [8.61616e-05, -0.0239314, 0.0210742, 8.07067e-05], 
 [0.0100089, -0.000320027, 8.07067e-05, 0.0100089]])

In [8]:
def is_symmetric(A, tol=1e-9):
    """Numerical symmetry test."""
    return float(ca.norm_inf(A - A.T)) < tol        # ‖A‑Aᵀ‖∞ < tol

def is_spd(A, sym_tol=1e-9, pd_tol=1e-12):
    """
    Symmetric‑positive‑definite test.
    1. Symmetrises small numerical noise:   A ← ½(A+Aᵀ)
    2. Tries a Cholesky factorisation.
    """
    A_sym = 0.5*(A + A.T)                         # cheap symmetrisation
    if not is_symmetric(A_sym, tol=sym_tol):
        return False

    try:
        ca.chol(A_sym + pd_tol*ca.DM.eye(A_sym.size1()))  # jitter protects near‑singular
        return True
    except RuntimeError:
        return False

In [9]:
is_spd(robot_inertia_matrix)

True